# Arch and Freq Invariance

This notebook shows the architecture and cpu frequency invariance features of EAS.

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [4]:
# Generate plots inline
%pylab inline

import json
import os

import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [5]:
# Setup a target configuration
my_target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    #"host"        : '192.168.1.101',
    "host"        : '10.169.36.83',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
    "rtapp-calib" : {
        '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    }

}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
         ],
         "buffsize" : 10240
    },

    "results_dir" : "arch_and_freq_invariance",
}

In [5]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

05:01:34  INFO    :         Target - Using base path: /data/lisa_fork/lisa
05:01:34  INFO    :         Target - Loading custom (inline) target configuration
05:01:34  INFO    :         Target - Loading custom (inline) test configuration
05:01:34  INFO    :         Target - External tools using:
05:01:34  INFO    :         Target -    ANDROID_HOME: /home/zhifei/zyang/android-sdk-linux
05:01:34  INFO    :         Target -    CATAPULT_HOME: /data/lisa_fork/lisa/tools/catapult
05:01:34  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
05:01:34  INFO    :         Target - Connecting Android target [192.168.1.101:5555]
05:01:34  INFO    :         Target - Connection settings:
05:01:34  INFO    :         Target -    {'device': '192.168.1.101:5555'}
05:01:36  INFO    :         Target - Initializing target workdir:
05:01:36  INFO    :         Target -    /data/local/tmp/devlib-target
05:01:43  INFO    :         Target - Topology:
05:01:43  INFO    :         Target -

# Workload configuration

In [6]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        
        'task_p50': Periodic(
            period_ms=400,         # period
            duty_cycle_pct=50,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.bigs[0])   # pinned on first big CPU
        ).get(),
        
        'task_r20': Ramp(
            start_pct=5,           # intial load
            end_pct=95,            # end load
            delta_pct=10,          # load % increase...
            time_s=1,              # ... every 1[s]            
        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

05:02:28  INFO    :          WlGen - Setup new workload simple
05:02:28  DEBUG   :          WlGen - Setup step [postrun] callback to [__postrun] function
05:02:28  DEBUG   :          WlGen - Configuring a profile-based workload...
05:02:28  DEBUG   :          RTApp - ref on big cpu: 1
05:02:28  INFO    :          RTApp - Workload duration defined by longest task
05:02:28  INFO    :          RTApp - Default policy: SCHED_OTHER
05:02:28  INFO    :          RTApp - ------------------------
05:02:28  INFO    :          RTApp - task [task_p50], sched: using default policy
05:02:28  INFO    :          RTApp -  | calibration CPU: 1
05:02:28  INFO    :          RTApp -  | loops count: 1
05:02:28  INFO    :          RTApp -  | CPUs affinity: 1
05:02:28  INFO    :          RTApp - + phase_000001: duration 10.000000 [s] (25 loops)
05:02:28  INFO    :          RTApp - |  period   400000 [us], duty_cycle  50 %
05:02:28  INFO    :          RTApp - |  run_time 200000 [us], sleep_time 200000 [us]
05:0

# Workload execution round 1

In [7]:
# Set performance governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('performance')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

05:02:56  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/shutils cpufreq_set_all_governors performance' | su"
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'"
05:02:56  INFO    : Target current governor: performance
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq'"
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'"
05:02:56  INFO    : Target big CPU max CPUfreq:850000
		    Target big CPU current CPUfreq: 850000
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq'"
05:02:56  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'"
05:02:56  INFO    : Target LITTLE CPU max CPUfreq:1100

In [8]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

05:03:10  INFO    : #### Setup FTrace
05:03:10  DEBUG   : adb -s 192.168.1.101:5555 shell "echo 'echo 10240 > '\\''/sys/kernel/debug/tracing/buffer_size_kb'\\''' | su"
05:03:10  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/kernel/debug/tracing/buffer_size_kb'"
05:03:10  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/trace-cmd reset' | su"
05:03:11  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/trace-cmd start -e cpu_frequency -e sched_load_avg_cpu -e sched_load_avg_task -e sched_switch -e cpu_capacity -e sched_blocked_reason -e sched_boost_cpu -e sched_boost_task -e sched_contrib_scale_f -e sched_cpu_hotplug -e sched_energy_diff -e sched_kthread_stop -e sched_kthread_stop_ret -e sched_load_avg_cpu -e sched_load_avg_task -e sched_migrate_task -e sched_move_numa -e sched_pi_setprio -e sched_process_exec -e sched_process_exit -e sched_process_fork -e sched_process_free -e sched_process_wait -e sched_stat_blocked -e sched_stat_iowait -e 

# Collected results

In [9]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

05:04:07  INFO    : Content of the output folder /data/lisa_fork/lisa/results/arch_and_freq_invariance


total 5048
drwxrwxr-x  2 zhifei zhifei    4096 4月  13 17:03 .
drwxrwxr-x 30 zhifei zhifei    4096 4月  13 17:01 ..
-rw-rw-r--  1 zhifei zhifei      67 4月  13 17:03 energy.json
-rw-rw-r--  1 zhifei zhifei     366 4月  13 17:03 output.log
-rw-rw-r--  1 zhifei zhifei    1075 4月  13 17:03 platform.json
-rw-r--r--  1 zhifei zhifei    3260 4月  13 17:03 rt-app-task_p50-0.log
-rw-r--r--  1 zhifei zhifei   12560 4月  13 17:03 rt-app-task_r20-1.log
-rw-r--r--  1 zhifei zhifei    3265 4月  13 17:03 simple_00.json
-rw-r--r--  1 zhifei zhifei 5124096 4月  13 17:03 trace.dat


In [10]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

05:04:17  INFO    : Generated RTApp JSON file:


In [11]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

05:04:22  INFO    : Energy: /data/lisa_fork/lisa/results/arch_and_freq_invariance/energy.json


{
    "LITTLE": 1.8965110000000038, 
    "big": 6.589620999999994
}


In [12]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

05:04:59  INFO    : Platform description: /data/lisa_fork/lisa/results/arch_and_freq_invariance/platform.json


# Trace inspection

In [13]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [15]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

05:06:13  DEBUG   : Loading [sched] events from trace in [/data/lisa_fork/lisa/results/arch_and_freq_invariance]...
05:06:13  DEBUG   : Parsing events: ['cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_switch', 'dequeue_task_fair', 'enqueue_task_fair', 'set_next_entity', 'cpu_capacity', 'sched_blocked_reason', 'sched_boost_cpu', 'sched_boost_task', 'sched_contrib_scale_f', 'sched_cpu_hotplug', 'sched_energy_diff', 'sched_kthread_stop', 'sched_kthread_stop_ret', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_migrate_task', 'sched_move_numa', 'sched_pi_setprio', 'sched_process_exec', 'sched_process_exit', 'sched_process_fork', 'sched_process_free', 'sched_process_wait', 'sched_stat_blocked', 'sched_stat_iowait', 'sched_stat_runtime', 'sched_stat_sleep', 'sched_stat_wait', 'sched_stick_numa', 'sched_swap_numa', 'sched_switch', 'sched_tune_boostgroup_update', 'sched_tune_config', 'sched_tune_tasks_update', 'sched_tune_filter', 'sched_wait_task', 'sched_wake_idle_wi

# Workload Execution Round 2

In [ ]:
# Set userspace governor with lowest CPUfreq
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('userspace')

target.write_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed", "450000")
target.write_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_setspeed", "450000")

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU min CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU min CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

05:06:28  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/shutils cpufreq_set_all_governors userspace' | su"
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "echo 'echo 450000 > '\\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'\\''' | su"
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'"
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "echo 'echo 450000 > '\\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_setspeed'\\''' | su"
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_setspeed'"
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'"
05:06:28  INFO    : Target current governor: userspace
05:06:28  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/sc

In [15]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

03:36:05  INFO    : #### Setup FTrace
03:36:05  DEBUG   : adb -s 192.168.1.101:5555 shell "echo 'echo 10240 > '\\''/sys/kernel/debug/tracing/buffer_size_kb'\\''' | su"
03:36:06  DEBUG   : adb -s 192.168.1.101:5555 shell "cat '/sys/kernel/debug/tracing/buffer_size_kb'"
03:36:06  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/trace-cmd reset' | su"
03:36:07  DEBUG   : adb -s 192.168.1.101:5555 shell "echo '/data/local/tmp/bin/trace-cmd start -e cpu_frequency -e sched_load_avg_cpu -e sched_load_avg_task -e sched_switch -e cpu_capacity -e sched_blocked_reason -e sched_boost_cpu -e sched_boost_task -e sched_contrib_scale_f -e sched_cpu_hotplug -e sched_energy_diff -e sched_kthread_stop -e sched_kthread_stop_ret -e sched_load_avg_cpu -e sched_load_avg_task -e sched_migrate_task -e sched_move_numa -e sched_pi_setprio -e sched_process_exec -e sched_process_exit -e sched_process_fork -e sched_process_free -e sched_process_wait -e sched_stat_blocked -e sched_stat_iowait -e 

In [16]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

03:36:32  INFO    : Content of the output folder /data/lisa_fork/lisa/results/arch_and_freq_invariance


total 16860
drwxrwxr-x  2 zhifei zhifei    4096 4月  13 15:36 .
drwxrwxr-x 30 zhifei zhifei    4096 4月  13 15:15 ..
-rw-rw-r--  1 zhifei zhifei      68 4月  13 15:36 energy.json
-rw-rw-r--  1 zhifei zhifei     366 4月  13 15:36 output.log
-rw-rw-r--  1 zhifei zhifei    1075 4月  13 15:36 platform.json
-rw-r--r--  1 zhifei zhifei    3260 4月  13 15:36 rt-app-task_p50-0.log
-rw-r--r--  1 zhifei zhifei   12560 4月  13 15:36 rt-app-task_r20-1.log
-rw-r--r--  1 zhifei zhifei    3265 4月  13 15:36 simple_00.json
-rw-r--r--  1 zhifei zhifei 6496256 4月  13 15:36 trace.dat
-rw-rw-r--  1 zhifei zhifei 5465764 4月  13 15:35 trace.raw.txt
-rw-rw-r--  1 zhifei zhifei 5252393 4月  13 15:35 trace.txt


# Collected Results

# Trace inspection

In [17]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [18]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

03:37:27  DEBUG   : Loading [sched] events from trace in [/data/lisa_fork/lisa/results/arch_and_freq_invariance]...
03:37:27  DEBUG   : Parsing events: ['cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_switch', 'dequeue_task_fair', 'enqueue_task_fair', 'set_next_entity', 'cpu_capacity', 'sched_blocked_reason', 'sched_boost_cpu', 'sched_boost_task', 'sched_contrib_scale_f', 'sched_cpu_hotplug', 'sched_energy_diff', 'sched_kthread_stop', 'sched_kthread_stop_ret', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_migrate_task', 'sched_move_numa', 'sched_pi_setprio', 'sched_process_exec', 'sched_process_exit', 'sched_process_fork', 'sched_process_free', 'sched_process_wait', 'sched_stat_blocked', 'sched_stat_iowait', 'sched_stat_runtime', 'sched_stat_sleep', 'sched_stat_wait', 'sched_stick_numa', 'sched_swap_numa', 'sched_switch', 'sched_tune_boostgroup_update', 'sched_tune_config', 'sched_tune_tasks_update', 'sched_tune_filter', 'sched_wait_task', 'sched_wake_idle_wi